# Data III


## Probleemstelling

Netflix maakt regelmatig beslissingen over de toekomst van series en films. Een productie kan soms meerdere seizoenen of vervolgfilms in de planning hebben staan, maar toch worden die dan geannuleerd. Dan kan de vraag ontstaan of het annuleren wel de juiste keuze was. 

Het annuleren van een vervolg op een film of serie wordt negatief ervaren door fans en kan de reputatie van Netflix verminderen, wat kan leiden tot minder abonnees en lagere kijkcijfers doordat niemand aan nieuwe shows of films begint.

Door middel van een onderzoek naar kijkcijfers, reviews en ratings van de eerdere delen, en reacties van het publiek wil Netflix erachter komen of het annuleren de juiste beslissing was. De taak van het machine learning team is om de reviews om te werken naar sentimenten om deze te kunnen gebruiken in een machine learning model. En een dashboard te maken waarmee de statistieken van een extra seizoen voorspeld kunnen worden.

#### Zakelijk inzicht  
Met behulp van dit onderzoek kan er worden gekeken naar de mogelijke performance van een volgend seizoen of vervolgfilm. Dit kan Netflix ondersteunen met onderbouwde keuzes maken, richting het annuleren van vervolgen om fans en abonnees tevreden te houden. Door te kijken naar de sentimenten, ratings en kijkcijfers kunnen zowel de bedrijfswinst en reputatie bij abonnees worden gewaarborgd. 

#### Literatuur onderzoek
Het onderzoek richt zich op sentiment analyse met behulp van Natural Language Processing (NLP), hiermee kunnen reviews van rotten tomatoes worden omgezet naar sentimenten. Na onderzoek is geen soort gelijk project waarmee het onderzoek vergeleken kan worden. 

Netflix is een online streaming service, via deze service kan eigen media en ook overgekochte media worden gestreamd. In het geval van dit Onderzoek zal het over de eigen media gaan, de Netflix Orginals. De Netflix Originals worden gemaakt door de Netflix studios, bij hen ligt dus ook de keuzes om deze te annuleren of vervolgen. 

#### vereisten informatie
- Een database met informatie en details van films en series van Netflix media.
- De belangrijke details zijn onder andere aantal seizoenen, aantal kijkers, release jaar, ratings, genre, regisseur, cast.
- De ratings moeten bruikbaar zijn bij machine learning.
- De tweets moeten omgezet worden naar een formaat dat bruikbaar is zodat een machine learning model er op kan trainen.
- De Rotten tomatoes website moet kunnen worden gescrapped 
- De API moet gebonden worden aan titels in de dataset

#### Databronnen
- Netflix series and movies Dataset: Netflix movies and TV shows. (2021, September 27). https://www.kaggle.com/datasets/shivamb/netflix-shows/data
- Rotten tomatoes: Rotten Tomatoes: movies | TV shows | Movie trailers | reviews. (n.d.). Rotten Tomatoes. https://www.rottentomatoes.com/
- API TMDB Movie database: Getting started. (n.d.). The Movie Database (TMDB). https://developer.themoviedb.org/reference/intro/getting-started

#### Data verwerken
- Dataset inladen naar DataFrame en klaar maken voor machine learning. Het verwijderen van de descriptions van de show/film en kolommen omzetten naar nummeriek bij bijvoorbeeld age rating een schaal van 1-5 maken.
- De rotten tomatoes site wordt gescrapped voor reviews en ratings per show/film. Bij shows wordt ook gekeken naar het meest recente seizoen
- Van TMDB wordt de data verzameld door middel van de API's

#### Soort data
- Dataset, gestructureerd: Er wordt gewerkt met een csv bestand waar elke rij een andere film/show betekent. 
- De rotten tomatoes, semi gestructureerd: De reviews kunnen worden gezien als ongestructureerd en de ratings als gestructureerd. Hierbij, dus de hele bron als semi gestructureerde data.
- TMDB, semi gestructureerd: De informatie wordt behaald door API's te gebruiken. Deze API's worden aangeboden in een gestandaardiseerde vorm. Toch, wordt er voor Semi gestructureerd gekozen vanwege de gestructureerde en ongestructureerde variabele van dit format.

## Data extraheren

### Database netflix films en series

In [6]:
pip install pandas sqlite3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [20]:
# importeren van de benodigde libraries
import pandas as pd
import sqlite3
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import logging
import matplotlib.pyplot as plt
import seaborn as sns

DEBUG: matplotlib data path: c:\Users\Miche\mapvoornotebook\Lib\site-packages\matplotlib\mpl-data
DEBUG: CONFIGDIR=C:\Users\Miche\.matplotlib
DEBUG: interactive is False
DEBUG: platform is win32
DEBUG: CACHEDIR=C:\Users\Miche\.matplotlib
DEBUG: Using fontManager instance from C:\Users\Miche\.matplotlib\fontlist-v330.json


In [8]:
netflix_k = pd.read_csv('netflix_titles.csv')
netflix_k = netflix_k.head(1000)
netflix_k
print(netflix_k['type'].value_counts())
display(netflix_k.head(10))


Movie      655
TV Show    345
Name: type, dtype: int64


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [9]:
def clean_titles(title):
    # Vervang speciale tekens
    title = title.replace("&", "and").replace(":", "")
    # Zet om naar kleine letters en vervang spaties door underscores
    title = title.lower().replace(" ", "_")
    return title


In [10]:

def get_url(row):
    clean_title = clean_titles(row['title'])
    
    if row['type'] == 'Movie':
        return f"m/{clean_title}"
    elif row['type'] == 'TV Show':
         if "Season" in row["duration"]:  
                season_number = int(row["duration"].split(" ")[0]) 
                season_formatted = f"/s{season_number:02}" 
                return f"/tv/{clean_title}{season_formatted}"


In [11]:
netflix_k['title'] = netflix_k['title'].apply(clean_titles)
netflix_k['url'] = netflix_k.apply(get_url, axis=1)

In [12]:
netflix_k

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,url
0,s1,Movie,dick_johnson_is_dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",m/dick_johnson_is_dead
1,s2,TV Show,blood_and_water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",/tv/blood_and_water/s02
2,s3,TV Show,ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,/tv/ganglands/s01
3,s4,TV Show,jailbirds_new_orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",/tv/jailbirds_new_orleans/s01
4,s5,TV Show,kota_factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,/tv/kota_factory/s02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,s996,Movie,vizontele,"Yılmaz Erdoğan, Ömer Faruk Sorak","Yılmaz Erdoğan, Demet Akbağ, Altan Erkekli, Ce...",Turkey,"April 23, 2021",2001,TV-MA,106 min,"Comedies, Dramas, International Movies","In 1974, a rural town in Anatolia gets its fir...",m/vizontele
996,s997,Movie,homunculus,Takashi Shimizu,"Go Ayano, Ryo Narita, Yukino Kishii, Anna Ishi...",Japan,"April 22, 2021",2021,TV-MA,116 min,"Horror Movies, International Movies, Thrillers",Truth and illusion blurs when a homeless amnes...,m/homunculus
997,s998,TV Show,life_in_color_with_david_attenborough,NaN,David Attenborough,"Australia, United Kingdom","April 22, 2021",2021,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...","Using innovative technology, this docuseries e...",/tv/life_in_color_with_david_attenborough/s01
998,s999,Movie,searching_for_sheela,NaN,Ma Anand Sheela,India,"April 22, 2021",2021,TV-14,58 min,"Documentaries, International Movies",Journalists and fans await Ma Anand Sheela as ...,m/searching_for_sheela


### Webscrapen van rotten tomatoes

In [19]:
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

all_reviews = {}
# Loop through each title to scrape reviews
for title in netflix_k['url']:
    url = f"https://www.rottentomatoes.com/{title}/reviews?type=user"  # Update URL format as needed
    logging.info(f"Scraping reviews for '{title}' from {url}")
    
    try:
        response = requests.get(url)
        time.sleep(0.5)  # Add a delay to prevent overwhelming the server

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all review paragraphs with the specified class
            reviews = soup.find_all('p', class_='audience-reviews__review js-review-text')
            stars = soup.find_all('rating-stars-group')  # Find all star ratings
            
            # Store the reviews and ratings in lists
            title_reviews = []
            title_ratings = []
            
            # Get up to 3 reviews and corresponding stars
            for index, review in enumerate(reviews[:3], start=1):  # Limit to 3 reviews
                title_reviews.append(review.get_text(strip=True))
                
                # Get the star rating for the review
                if index - 1 < len(stars):
                    star_score = stars[index - 1].get('score', 'N/A')  # Get 'score' attribute
                    title_ratings.append(float(star_score))
                else:
                    title_ratings.append("N/A")  # No stars available

            # Store in the main dictionary
            all_reviews[title] = {
                "reviews": title_reviews,
                "ratings": title_ratings,
            }
            logging.info(f"Successfully scraped reviews for '{title}'")
        else:
            logging.error(f"Failed to load the page for '{title}'")
            all_reviews[title] = {
                "reviews": ["Error: Failed to load the page."],
                "ratings": [],
            }
    except Exception as e:
        logging.error(f"An error occurred for '{title}': {str(e)}")
        all_reviews[title] = {
            "reviews": ["Error: An exception occurred."],
            "ratings": [],
        }

# Output the reviews and ratings
for title, data in all_reviews.items():
    print(f"\nReviews for '{title}':")
    for i, (review, rating) in enumerate(zip(data["reviews"], data["ratings"]), start=1):
        print(f"Review {i}: {review}")
        print(f"Rating: {rating}\n" + ("-" * 40))



#https://chatgpt.com/share/677d2ba4-37ec-8010-bd07-9c8aacb3b740 
# duur = 9 minuten 14 seconden zonder aanpassingen
# duur = 42 minuten 56 seconden met aanpassingen (sleep(2) en logging)
# duur = 18 minuten 35 seconden met aanpassingen (sleep(0.5) en logging) 
# Met selenium krijg je en geen sterren en duurt het 66 min

INFO: Scraping reviews for 'm/dick_johnson_is_dead' from https://www.rottentomatoes.com/m/dick_johnson_is_dead/reviews?type=user


DEBUG: Starting new HTTPS connection (1): www.rottentomatoes.com:443
DEBUG: https://www.rottentomatoes.com:443 "GET /m/dick_johnson_is_dead/reviews?type=user HTTP/1.1" 200 36728
INFO: Successfully scraped reviews for 'm/dick_johnson_is_dead'
INFO: Scraping reviews for '/tv/blood_and_water/s02' from https://www.rottentomatoes.com//tv/blood_and_water/s02/reviews?type=user
DEBUG: Starting new HTTPS connection (1): www.rottentomatoes.com:443
DEBUG: https://www.rottentomatoes.com:443 "GET //tv/blood_and_water/s02/reviews?type=user HTTP/1.1" 200 27073
INFO: Successfully scraped reviews for '/tv/blood_and_water/s02'
INFO: Scraping reviews for '/tv/ganglands/s01' from https://www.rottentomatoes.com//tv/ganglands/s01/reviews?type=user
DEBUG: Starting new HTTPS connection (1): www.rottentomatoes.com:443
DEBUG: https://www.rottentomatoes.com:443 "GET //tv/ganglands/s01/reviews?type=user HTTP/1.1" 200 27662
INFO: Successfully scraped reviews for '/tv/ganglands/s01'
INFO: Scraping reviews for '/tv/


Reviews for 'm/dick_johnson_is_dead':
Review 1: This is a documentary that will hit many different people in many different ways depending on their past and present life experiences. Without getting too personal, in some ways I am living out a bit of what is happening in this movie and found parallels that are eerily accurate, including a need to help figure out the TV that happened just a few hours earlier today. As a result, this was a rather extraordinary experience for me. I'll be honest - when I first read the description of this movie, it sounded a bit off kilter, and I think it could have gone terribly wrong in numerous ways. But director/daughter Johnson is a smart cookie and finds a beautiful balance between the pain and the humor. While I expected to be moved - and I most certainly was - I was surprised by how often I smiled and laughed throughout. And it helps that Dick Johnson is just the loveliest man ever, a kind and gentle soul whose smile lights up a room and whose spi

## Uitleg over het web scrapen:
Dit script verzamelt gebruikersbeoordelingen en sterrenbeoordelingen van films of series op **Rotten Tomatoes**. Het maakt gebruik van de `requests`-bibliotheek om webpagina's op te halen, `BeautifulSoup` om de HTML van de pagina te parseren, en `logging` om meldingen en fouten te loggen.

## Stappen van het Script

1. **Logging Configuratie:**    
   Het script configureert logging om meldingen van het type **DEBUG** of hoger weer te geven. Dit helpt bij het volgen van de voortgang en het opsporen van fouten:      

   ```python
   logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)
2. **Dictionary voor reviews:** 
    De variabele all_reviews is een lege dictionary die de beoordelingen en sterrenbeoordelingen van elke titel opslaat:    
    all_reviews = {}
3. **Loop door de titels:** 
    Het script doorloopt de lijst van URL's van films/series die moeten worden gescrapet. Elke URL vertegenwoordigt een specifieke titel:   
    for title in netflix_k['url']:
4. **Opbouw van de URl en Logging:**    
    De URL voor elke titel wordt opgebouwd en een log bericht wordt gegenereerd om aan te geven dat het script de beoordelingen van die specifieke titel aan het scrapen is:    
    url = f"https://www.rottentomatoes.com/{title}/reviews?type=user"   
    logging.info(f"Scraping reviews for '{title}' from {url}")
5. **Verzoek naar de pagina:**  
    Een GET-verzoek wordt gedaan naar de URL en er wordt een vertraging van 0.5 seconden toegevoegd om de server niet te overbelasten:  
    response = requests.get(url)
    time.sleep(0.5)
6. **Controle op de status van het verzoek:**   
    Het script controleert of de statuscode van de pagina 200 is, wat betekent dat de pagina succesvol is geladen:  
    if response.status_code == 200:
7. **HTML parseren met BeautifulSoup:** 
    De HTML van de pagina wordt geparsed met BeautifulSoup om de relevante informatie (beoordelingen en sterren) eruit te extraheren:   
    soup = BeautifulSoup(response.content, 'html.parser')
8. **Beoordelingen en sterren extraheren:** 
    De tekst van de recensies wordt opgehaald via de p-tags met de class 'audience-reviews__review js-review-text'. De sterrenbeoordelingen worden opgehaald uit de 'rating-stars-group'-tags:      
    reviews = soup.find_all('p', class_='audience-reviews__review js-review-text')  
    stars = soup.find_all('rating-stars-group')
9. **Beoordelingen en sterren opslaan:**    
    Het script haalt maximaal 3 beoordelingen per titel op, evenals de bijbehorende sterrenbeoordelingen. Als er geen sterinformatie is, wordt "N/A" toegevoegd:    
    for index, review in enumerate(reviews[:3], start=1):   
    title_reviews.append(review.get_text(strip=True))   
    if index - 1 < len(stars):  
        star_score = stars[index - 1].get('score', 'N/A')   
        title_ratings.append(float(star_score)) 
    else:   
        title_ratings.append("N/A") 
10. **Opslaan in de 'all_reviews' dictionary:** 
    De verzamelde beoordelingen en sterren worden opgeslagen in de all_reviews dictionary:  
    all_reviews[title] = {      
    "reviews": title_reviews,   
    "ratings": title_ratings,}  
11. **Fouthandeling:**  
    Als er een fout optreedt tijdens het scrapen, wordt deze geregistreerd in de log en wordt een foutmelding opgeslagen in de reviews en ratings voor die titel:   
    except Exception as e:  
    logging.error(f"An error occurred for '{title}': {str(e)}") 
    all_reviews[title] = {  
        "reviews": ["Error: An exception occurred."],   
        "ratings": [],  
    }
12. **Weergave van de verzamelde gegevens:**    
    Na het scrapen worden de beoordelingen en sterren voor elke titel afgedrukt:    
    for title, data in all_reviews.items(): 
    print(f"\nReviews for '{title}':")  
    for i, (review, rating) in enumerate(zip(data["reviews"], data["ratings"]), start=1):   
        print(f"Review {i}: {review}")      
        print(f"Rating: {rating}\n" + ("-" * 40))   


## Wat er gebeurd
Deze code haalt de gebruikersbeoordelingen en sterrenbeoordelingen van films en series op via de site van **Rotten Tomatoes**. Hiervoor word er gebruikgemaakt van de `request` library om webpagina's op te galen, en `BeautifulSoup` om de HTML van de website te analyseren. Om de fouten en voortgang van de code bij te houden, is er gebruikgemaakt van logging.

Allereerst word er van te voren een lijst gemaakt met de titels van de films en series, in een specifiek formaat dat in de url van een website past. Omdat Rotten Tomatoes een verschillende URL structuur heeft voor films en series, worden de titels in de lijst hierop aangepast. Voor films wordt de URL aangepast met `/m`, en voor series met `/tv`.

Wanneer de URL correct is opgebouwd, word de lijst in de for-loop gezet en wordt er een GET-verzoek gestuurd naar de pagina met de reviews. Er is een vertraging van 0.5 seconden ingebouwd tussen de verzoeken, om te voorkomen dat de server overbelast raakt. Als de pagina word geladen controleert het script of de statuscode 200 is. Hierna word de HTML van de pagina geparsed met `BeautifulSoup`.

De beoordelingen worden verzameld uit de `<p>` tag m.et de class `audience-reviews__review js-review-text`. De sterren worden uit de `rating-stars-group` tag gehaald. Om de belasting van de server te verminderen, en de draai tijd van de code te verminderen, worden er max drie reviews per titel opgehaald. Als er geen sterrenbeoordeling beschikbaar is, word er 'N/A' ingevuld.

De code houdt rekening met eventuele foutmeldingen. Wanneer een film of serie niet op Rotten Tomatoes staat, of als er geen beoordelingen beschikbaar zijn, wordt dit gelogd. Zo is het makkelijker te achterhalen waar iets misgaat en kan er gecontroleerd worden of de fout in de code of de website ligt.

Tot slot worden de reviews en sterren overzichtelijk geprint. Dit maakt het makkelijk om de resultaten te analyseren, en eventuele fouten sneller op te kunnen lossen. Daarnaast geeft het snel een goede indruk of de code werkt.

### API 

In [21]:
API_KEY = '1d4b0a66740e090a9d5bedddcdeaa0e6'
BASE_URL = "https://api.themoviedb.org/3"

In [22]:
def get_netflix_shows():
    """this function get basic data about all netflix shows from the API

    Returns
    -------
    netflix_shows : list
        a list of all netflix shows where every item is a dictionary containing basic data about the show
    """    
    url = f"{BASE_URL}/discover/tv"
    netflix_shows = []
    page = 1

    # De code blijft loopen totdat er geen nieuwe pagina's meer zijn
    while True:
        params = {
            "api_key": API_KEY,
            "with_networks": 213,  # Omdat we specifiek onderzoek doen naar Netflix shows, gebruiken we standaard het id van Netflix
            "page": page
        }
        response = requests.get(url, params=params)
        data = response.json()
        netflix_shows.extend(data['results'])

        # Als er geen nieuwe pagina's meer zijn, wordt the loop onderbroken.
        if page >= data['total_pages']:
        # if page >= 20: # Puur voor testen, alles laden duurt lang
            break
        page += 1

    return netflix_shows


def get_show_data(series_id, new_data):
    """this function get specifiic data about shows from the API

    Parameters
    ----------
    series_id : int
        id which refers to a show in themoviedb
    new_data : str
        decides what data will be requested from the API, name must match the API

    Returns
    -------
    json
        the requested data from the API
    
    Raises
    ------
    ValueError
        if the new_data doesn't match an entry the API
    """   

    url = f"{BASE_URL}/tv/{series_id}"
    params = {"api_key": API_KEY}
    response = requests.get(url, params=params)

    try:
        return response.json().get(new_data)
    except:
        raise ValueError(f"{new_data} komt niet voor in de api, check voor mogelijke opties op: https://developer.themoviedb.org/reference/tv-series-details")
    


def get_season_data(series_id, season_number, new_data):
    """this function gets specific data from a chosen season

    Parameters
    ----------
    series_id : int
        the id of the show in themoviedb
    season_number : int
        the number of the season you want to get data from
    new_data : str
        decides what data will be requested from the API, name must match the API

    Returns
    -------
    json
        the requested data from the API

    Raises
    ------
    ValueError
        if the new_data doesn't match an entry the API
    """    
    # check de status in de df want returning series hebben vaak een nog niet beoordeeld seizoen.
    url = f"{BASE_URL}/tv/{series_id}/season/{season_number}"
    params = {"api_key": API_KEY}
    response = requests.get(url, params=params)

    try:
        return response.json().get(new_data)
    except:
        raise ValueError(f"{new_data} komt niet voor in de api, check voor mogelijke opties op: https://developer.themoviedb.org/reference/tv-seasons")

In [23]:
netflix_shows = get_netflix_shows()

DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/discover/tv?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6&with_networks=213&page=1 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/discover/tv?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6&with_networks=213&page=2 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/discover/tv?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6&with_networks=213&page=3 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/discover/tv?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6&with_networks=213&page=4 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/discover/tv?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6&with

In [24]:
# hieronder is te zien welke attributen we gelijk al in kunnen laden, voor specifiekere data moeten we het id van een serie gebruiken als key voor de api.
netflix_shows[0]

{'adult': False,
 'backdrop_path': '/87mebbBtoWzHV0kILgV6M7yIfun.jpg',
 'genre_ids': [10759, 9648, 18],
 'id': 93405,
 'origin_country': ['KR'],
 'original_language': 'ko',
 'original_name': '오징어 게임',
 'overview': "Hundreds of cash-strapped players accept a strange invitation to compete in children's games. Inside, a tempting prize awaits — with deadly high stakes.",
 'popularity': 4965.829,
 'poster_path': '/1QdXdRYfktUSONkl1oD5gc6Be0s.jpg',
 'first_air_date': '2021-09-17',
 'name': 'Squid Game',
 'vote_average': 7.9,
 'vote_count': 15046}

In [25]:
# De volgorde van de kolommen in het dataframe wordt hiermee bepaald. Maakt alles wat overzichtelijker door vergelijkbare kolommen naast elkaar te plaatsen.
columnList = ['name', 'id', 'number_of_seasons', 'status', 'vote_average', 'final_season_rating']

# vote_average wordt hernoemd naar average_series_rating voor consistentie in benaming.
df = pd.DataFrame(netflix_shows, columns=columnList).rename(columns={'vote_average': 'average_series_rating'})

df['status'] = df['id'].apply(get_show_data, new_data='status')
df['number_of_seasons'] = df['id'].apply(get_show_data, new_data='number_of_seasons')
df['final_season_rating'] = df.apply(lambda x: get_season_data(x.id, x.number_of_seasons, new_data='vote_average'), axis=1)

# Als er maar één seizoen is, wordt de final_season_rating gelijkgesteld aan de average_series_rating.
df.loc[df['number_of_seasons'] == 1, 'final_season_rating'] = df['average_series_rating']

# 'Returning Series' hebben vaak een nog niet beoordeeld seizoen. Loop hier nog een keer overheen en vervang de rating met die van het een-na-laatste seizoen.
df.loc[(df['status'] == 'Returning Series') & (df['final_season_rating'] == 0.0), 'final_season_rating'] = df.apply(lambda x: get_season_data(x.id, x.number_of_seasons - 1, new_data='vote_average'), axis=1)

# We ronden alle ratings op een enkel kommagetal af.
df = df.round(1)
df.head(3)

DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/tv/93405?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/tv/50524?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/tv/63174?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/tv/215866?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org:443 "GET /3/tv/110316?api_key=1d4b0a66740e090a9d5bedddcdeaa0e6 HTTP/1.1" 200 None
DEBUG: Starting new HTTPS connection (1): api.themoviedb.org:443
DEBUG: https://api.themoviedb.org

,name,id,number_of_seasons,status,average_series_rating,final_season_rating
0,Squid Game,93405,3,Returning Series,7.9,8.6
1,Pen Tor,50524,12,Returning Series,8.8,0.0
2,Lucifer,63174,6,Ended,8.5,7.9


In [ ]:
# Dit is handig om visualisaties mee te maken zonder eerst alle data opnieuw op te halen uit de api, duurt vrij lang namelijk.
api_data = df.to_csv('../api_data.csv')